### Importing the packages

In [1]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from itertools import chain
from nltk.tokenize import MWETokenizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest ,chi2
from sklearn.svm import LinearSVC
from sklearn import preprocessing
import numpy as np
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
from sklearn.linear_model import RidgeClassifier
#from sklearn.pipeline import Pipeline
#from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import  MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

In [3]:
from textblob import TextBlob
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.pipeline import Pipeline
#from sklearn.linear_model import SGDClassifier
#from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from collections import OrderedDict

In [31]:
#IMporting the training document
traindoc = open('C:/Users/mrutula1995/Documents/ada/data/training_docs.txt',mode = "r",encoding="utf-8-sig")
train = traindoc.readlines()
traindoc.close()

In [32]:
#Importing the test document
testdoc = open('C:/Users/mrutula1995/Documents/ada/data/testing_docs.txt',mode = "r",encoding="utf-8-sig")
test = testdoc.readlines()
testdoc.close()

In [33]:
#training labels
trainlabel = open('C:/Users/mrutula1995/Documents/ada/data/training_labels_final.txt',mode = 'r',encoding = "utf-8-sig")
trainLB = trainlabel.readlines()
trainlabel.close()

In [34]:
#Removing space from the labels
trainLB = [t.strip() for t in trainLB]
trainLB = [t.split()[1] for t in trainLB]

In [35]:
len(train)

425780

## Preprocessing

In [36]:
#obtaining the ID of the Train document 
i = 0
col_id = []         
while i < len(train):
    col_id.append(train[i])    
    i += 4

In [37]:
#obtaining the ID of the Test document 
i = 0
test_col_id = []         
while i < len(test):
    test_col_id.append(test[i])    
    i += 4

In [38]:
col_id = [x.strip() for x in col_id]     #to remove the \n in the end
col_id = [re.sub(r'^(ID\s)','',i) for i in col_id]  #Replacing 'ID' with empty character

In [39]:
test_col_id = [x.strip() for x in test_col_id]     #to remove the \n in the end
test_col_id = [re.sub(r'^(ID\s)','',i) for i in test_col_id] #Replacing 'ID' with empty character

In [40]:
#obtaining the txt of the Train document
i = 1
text = []           
while i < len(train):
    text.append(train[i])
    i += 4    

In [41]:
#obtaining the txt of Test the document
i = 1
text_test = []           
while i < len(test):
    text_test.append(test[i])
    i += 4    

In [42]:
#Removing the space from the train text 
text = [t.strip() for t in text]
#Replacing the word TEXT with empty character
text = [re.sub(r'^(TEXT)','',t) for t in text]

In [43]:
#Removing the space from the test text 
text_test = [t.strip() for t in text_test]
#Replacing the word TEXT with empty character
text_test = [re.sub(r'^(TEXT)','',t) for t in text_test]

In [44]:
#Creating train dataframe
trainDF = pd.DataFrame()
trainDF['ID'] = col_id
trainDF['text'] = text
trainDF['label'] = trainLB

In [45]:
#Creating test dataframe
testDF = pd.DataFrame()
testDF['ID'] = test_col_id
testDF['text'] = text_test
#testDF['label'] = testLB

In [46]:
trainDF = trainDF[trainDF.text != " '"]

In [47]:
#Preprocessing the train text
trainDF["text"].replace(r"http\S+", "URL", regex=True,inplace=True)
trainDF["text"].replace(r"@\S+", "REF", regex=True ,inplace=True)
trainDF["text"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
trainDF["text"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
trainDF["text"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
trainDF["text"].replace(r"\d+", "NUM", regex=True,inplace=True)

In [48]:
#Preprocessing the test text
#testDF['num_of_words'] = testDF["text"].str.split().apply(len)
testDF["text"].replace(r"http\S+", "URL", regex=True,inplace=True)
testDF["text"].replace(r"@\S+", "REF", regex=True ,inplace=True)
testDF["text"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
testDF["text"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
testDF["text"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
testDF["text"].replace(r"\d+", "NUM", regex=True,inplace=True)

### Stemming

In [49]:
#PorterStemmer object
porter_stemmer = PorterStemmer()
    
    #text_processed = [porter_stemmer.stem(word) for word in text_processed]

#Preprocessing function to stem the sentence
def clearstring(string):
    try:
        string = re.sub('[^A-Za-z .]+', '', string)
        string = string.split(' ')
        string = filter(None, string)
        string = [y.strip() for y in string]        
        string = [porter_stemmer.stem(word) for word in string]
        string = ' '.join(string)
    except:
        print(string)
    return string

In [50]:
for i in range(trainDF.shape[0]):
    trainDF['text'].iloc[i] = clearstring(trainDF['text'].iloc[i])

In [51]:
for i in range(testDF.shape[0]):
    testDF['text'].iloc[i] = clearstring(testDF['text'].iloc[i])

In [54]:
# I just want verbs and nouns in a sentence
def get_clean_text(string):
    blob = TextBlob(string).tags
    tags = []
    # you can add more
    accept = ['NNP', 'NN', 'NNS', 'NNPS', 'VBZ', 'VBN', 'VB']
    for k in blob:
        if k[1] in accept:
            tags.append(k[0])
            
    return list(OrderedDict.fromkeys(tags))

In [55]:
# we need to split by ('.') to save our memory during speech tagging process to build the tree
for i in range(trainDF.shape[0]):
    texts = trainDF['text'].iloc[i].split('. ')
    tags = []
    for t in texts: 
        tags += get_clean_text(t)
    trainDF['text'].iloc[i] = ' '.join(list(OrderedDict.fromkeys(tags)))

In [67]:
# we need to split by ('.') to save our memory during speech tagging process to build the tree
for i in range(testDF.shape[0]):
    texts = testDF['text'].iloc[i].split('. ')
    tags = []
    for t in texts:
        tags += get_clean_text(t)
    testDF['text'].iloc[i] = ' '.join(list(OrderedDict.fromkeys(tags)))

In [56]:
trainDF['text'].iloc[1]

'act drug dure home number oxblood is bag believ woman been face court morning varley time ha biproduct amphetamin manufactur process wherebi powder crystal produc methamphetamin phospher iodin colour'

In [68]:
testDF['text'].iloc[1]

'govern is queensland respons explain deni transfer alic spring prison tommi neale neal court challeng decision convict murder mount isa num request territori law serv life sentenc parole lawyer have been abl appli parol year kim aborigin aid ha stonewal effort find applic be wa reject attorney gener peter toyn correspond repres correct minist toni mcgradi refus becaus behaviour sustain famili grounds hi offic statement stands'

In [19]:
labels = trainDF['label'].values.copy()

# get unique label
unique_labels = np.unique(labels)
# change into int
labels = LabelEncoder().fit_transform(labels)
texts = trainDF['text'].values.copy()

In [ ]:
mapper = DataFrameMapper([
     ('text', TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=10000)),
     ('text', TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.8,max_features=10000))
    
 ],df_out=True)

In [ ]:
map_data = mapper.fit_transform(X.copy())

In [ ]:
list_mapper = mapper.transformed_names_

In [ ]:
list_mapper = [re.sub(r'text_','',l) for l in list_mapper]

In [57]:
tfidf_transformer_train =  TfidfVectorizer(analyzer='word',ngram_range=(1, 2),max_df=0.5,norm='l2',max_features=2000)
tfidf_train = tfidf_transformer_train.fit_transform(trainDF.text)

In [74]:
train_mat = pd.DataFrame(tfidf_train.toarray(), columns=tfidf_transformer_train.get_feature_names())

In [75]:
train_mat['label'] = trainDF['label']

In [60]:
train_mat.to_csv("H_train_data.csv",index=False,encoding="utf-8")

### TFIDVectorizer

In [69]:
tfidf_transformer_test =  TfidfVectorizer(analyzer='word',ngram_range=(1, 2),max_df=0.5,norm='l2',max_features=2000)
tfidf_test = tfidf_transformer_test.fit_transform(testDF.text)

In [81]:
test_mat = pd.DataFrame(tfidf_test.toarray(), columns=tfidf_transformer_test.get_feature_names())

In [82]:
test_name = list(test_mat)

In [ ]:
train_mat = train_mat.drop(labels = 'label',axis = 1)

In [85]:
train_name = list(train_mat)

In [90]:
test_name = pd.DataFrame()
for t in test_mat:
    #print(t)
    if t in train_name:
        test_name[t] = test_mat[t]
        
        

In [92]:
diff = [x for x in train_name if x not in test_mat]

In [95]:
diff[1]

'alloc'

In [98]:
for d in diff:
    test_name[d] = 0

In [100]:
test_name.to_csv("H_test_data.csv",index=False,encoding="utf-8")

In [ ]:
train_mat = pd.DataFrame(tfidf_train.todense(), index=data.index, columns=cv.get_feature_names())
train_mat['number'] = data.number
print(bigrams)

In [30]:
#SGD Classifier is used
magazine_clf = Pipeline([('vect', CountVectorizer()), 
                         ('tfidf', TfidfTransformer()), 
                         ('clf', SGDClassifier(loss = 'modified_huber', 
                                               penalty = 'l2', alpha = 1e-4, 
                                               n_iter = 100, random_state = 42))])

### Crossvalidation

In [31]:
x_train, x_test, y_train, y_test = train_test_split(texts, labels, test_size = 0.2)
magazine_clf.fit(x_train, y_train)
predicted = magazine_clf.predict(x_test)
print (np.mean(predicted == y_test))

C:\Users\mrutula1995\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.7756118183099253


In [34]:
magazine_clf = Pipeline([('vect',TfidfVectorizer(ngram_range = (1,2), stop_words = "english",max_df = 0.99,sublinear_tf= True)),
                         ('chi',SelectKBest(chi2,k=50000)),
                          ('clf', SGDClassifier(loss = 'modified_huber', 
                                               penalty = 'l2', alpha = 1e-4, 
                                               n_iter = 100, random_state = 42))])
                         

In [33]:
x_train, x_test, y_train, y_test = train_test_split(texts, labels, test_size = 0.2)
magazine_clf.fit(x_train, y_train)
predicted = magazine_clf.predict(x_test)
print (np.mean(predicted == y_test))

C:\Users\mrutula1995\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.7532528535863592


In [36]:
scores = cross_val_score(magazine_clf,texts,labels,cv=5,scoring='f1_micro')

C:\Users\mrutula1995\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\mrutula1995\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\mrutula1995\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\mrutula1995\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\mrutula1995\Anaconda3\lib\site-packages